In [1]:
# Import Modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import hvplot.pandas
import numpy as np
import tensorflow as tf
import sqlite3
import requests
from pathlib import Path
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
# Display all of the columns
pd.set_option('display.max_columns', None)

In [2]:
# Raw URL of the database file on GitHub
url = 'https://github.com/BFletchall/Project-4-Group-4-Machine-Learning/raw/main/claimed_time.db'
# Download the database file
r = requests.get(url)
with open('claimed_time.db', 'wb') as f:
    f.write(r.content)
# Connect to the SQLite database
conn = sqlite3.connect('claimed_time.db')
# Query data from the database table
query = "SELECT * FROM claimed_table"
# Read data into DataFrame
df = pd.read_sql(query, conn)
# Close the database connection
conn.close()
# Display the first few rows of the DataFrame
df.head()

,NumberWithDash,EPM_ProductionControlItemID,EPM_MainMark,EPM_InstanceNumber,Assembly_MainPieceProductionCode,Assembly_MainPartLengthFt,Assembly_WeightEachLbs,Assembly_SurfaceAreaEachSqFt,Assembly_MainPartShape,Assembly_MainPartDimension,Assembly_MainPartFinishDescr,Assembly_TotalQuantityInJob,Assembly_NumSmallParts,EPM_AdjustedStationName,TimeInSeconds_ThisWorkSegment,Assembly_EstTotalHours_ThisLaborGroup
0,19-048,870038,T124,1,S.S. TUBE STEEL,2.81,39.575,4.43,HSS,4 x 2 x 0.25,N - No Paint,1,1,Fab/Layout,142,0.0
1,19-048,870040,T125,1,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Fab/Layout,124,0.0
2,19-048,870040,T125,1,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Weld,47,0.0
3,19-048,870040,T125,2,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Fab/Layout,124,0.0
4,19-048,870040,T125,2,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Weld,48,0.0


In [3]:
df.columns

Index(['NumberWithDash', 'EPM_ProductionControlItemID', 'EPM_MainMark',
       'EPM_InstanceNumber', 'Assembly_MainPieceProductionCode',
       'Assembly_MainPartLengthFt', 'Assembly_WeightEachLbs',
       'Assembly_SurfaceAreaEachSqFt', 'Assembly_MainPartShape',
       'Assembly_MainPartDimension', 'Assembly_MainPartFinishDescr',
       'Assembly_TotalQuantityInJob', 'Assembly_NumSmallParts',
       'EPM_AdjustedStationName', 'TimeInSeconds_ThisWorkSegment',
       'Assembly_EstTotalHours_ThisLaborGroup'],
      dtype='object')

In [4]:
df_reduce =df.drop(columns=['EPM_MainMark', 'Assembly_EstTotalHours_ThisLaborGroup', 'NumberWithDash'], inplace=False)
df_reduce

,EPM_ProductionControlItemID,EPM_InstanceNumber,Assembly_MainPieceProductionCode,Assembly_MainPartLengthFt,Assembly_WeightEachLbs,Assembly_SurfaceAreaEachSqFt,Assembly_MainPartShape,Assembly_MainPartDimension,Assembly_MainPartFinishDescr,Assembly_TotalQuantityInJob,Assembly_NumSmallParts,EPM_AdjustedStationName,TimeInSeconds_ThisWorkSegment
0,870038,1,S.S. TUBE STEEL,2.81,39.575,4.43,HSS,4 x 2 x 0.25,N - No Paint,1,1,Fab/Layout,142
1,870040,1,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Fab/Layout,124
2,870040,1,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Weld,47
3,870040,2,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Fab/Layout,124
4,870040,2,S.S. TUBE STEEL,7.91,112.116,12.50,HSS,4 x 2 x 0.25,N - No Paint,2,1,Weld,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8402,876966,1,PIPE,8.03,16.440,7.73,API,1 1/4SCH40,L - Anodized Aluminum,1,3,Fab/Layout,16751
8403,876966,1,PIPE,8.03,16.440,7.73,API,1 1/4SCH40,L - Anodized Aluminum,1,3,Material Handling,17
8404,876966,1,PIPE,8.03,16.440,7.73,API,1 1/4SCH40,L - Anodized Aluminum,1,3,Weld,4153
8405,904279,1,PLATE,9.92,94.903,25.14,PL,3/16 x 15,"D - Steel, Paint All",1,0,Fab/Layout,9562


In [5]:
df_reduce.columns

Index(['EPM_ProductionControlItemID', 'EPM_InstanceNumber',
       'Assembly_MainPieceProductionCode', 'Assembly_MainPartLengthFt',
       'Assembly_WeightEachLbs', 'Assembly_SurfaceAreaEachSqFt',
       'Assembly_MainPartShape', 'Assembly_MainPartDimension',
       'Assembly_MainPartFinishDescr', 'Assembly_TotalQuantityInJob',
       'Assembly_NumSmallParts', 'EPM_AdjustedStationName',
       'TimeInSeconds_ThisWorkSegment'],
      dtype='object')

In [6]:
# Determine the number of unique values in each column.
unique_values = df_reduce.nunique()
print(unique_values)

EPM_ProductionControlItemID         2058
EPM_InstanceNumber                   669
Assembly_MainPieceProductionCode      58
Assembly_MainPartLengthFt            944
Assembly_WeightEachLbs              1642
Assembly_SurfaceAreaEachSqFt        1530
Assembly_MainPartShape                19
Assembly_MainPartDimension           340
Assembly_MainPartFinishDescr          11
Assembly_TotalQuantityInJob           58
Assembly_NumSmallParts                89
EPM_AdjustedStationName                8
TimeInSeconds_ThisWorkSegment       2670
dtype: int64


In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
df_reduce_encoded = pd.get_dummies(df_reduce)
df_reduce_encoded

,EPM_ProductionControlItemID,EPM_InstanceNumber,Assembly_MainPartLengthFt,Assembly_WeightEachLbs,Assembly_SurfaceAreaEachSqFt,Assembly_TotalQuantityInJob,Assembly_NumSmallParts,TimeInSeconds_ThisWorkSegment,Assembly_MainPieceProductionCode_ALUM ANGLE,Assembly_MainPieceProductionCode_ALUM BEAM,Assembly_MainPieceProductionCode_ALUM CHANNEL,Assembly_MainPieceProductionCode_ALUM CHKD PLATE,Assembly_MainPieceProductionCode_ALUM EMBED ANGLE,Assembly_MainPieceProductionCode_ALUM PIPE,Assembly_MainPieceProductionCode_ALUM PIPE LADDER,Assembly_MainPieceProductionCode_ALUM PIPE SLEEVE,Assembly_MainPieceProductionCode_ALUM PLATE,Assembly_MainPieceProductionCode_ALUM STRINGER,Assembly_MainPieceProductionCode_ALUM TUBE COLUMN,Assembly_MainPieceProductionCode_ANGLE,Assembly_MainPieceProductionCode_BEAM,Assembly_MainPieceProductionCode_BENT PLATE,Assembly_MainPieceProductionCode_BOLLARD,Assembly_MainPieceProductionCode_BRACE,Assembly_MainPieceProductionCode_BRACKET,Assembly_MainPieceProductionCode_Backer Bar,Assembly_MainPieceProductionCode_CHANNEL,Assembly_MainPieceProductionCode_CHK PLATE,Assembly_MainPieceProductionCode_COLUMN,Assembly_MainPieceProductionCode_CRANE GIRDER,Assembly_MainPieceProductionCode_DOOR PLATE,Assembly_MainPieceProductionCode_EMBED,Assembly_MainPieceProductionCode_EMBED ANGLE,Assembly_MainPieceProductionCode_EMBED PLATE,Assembly_MainPieceProductionCode_FRAME,Assembly_MainPieceProductionCode_FRAME_COLUMN,Assembly_MainPieceProductionCode_GALV. STRINGER,Assembly_MainPieceProductionCode_GUSSET PL,Assembly_MainPieceProductionCode_H BRACE,Assembly_MainPieceProductionCode_HSS STRUT,Assembly_MainPieceProductionCode_PIPE,Assembly_MainPieceProductionCode_PLATE,Assembly_MainPieceProductionCode_PLATE WASHER,Assembly_MainPieceProductionCode_RAIL,Assembly_MainPieceProductionCode_ROLLED TUBE STEEL,Assembly_MainPieceProductionCode_S.S. ANGLE,Assembly_MainPieceProductionCode_S.S. BENT PLATE,Assembly_MainPieceProductionCode_S.S. EMBED ANGLE,Assembly_MainPieceProductionCode_S.S. PLATE,Assembly_MainPieceProductionCode_S.S. PLATE LADDER,Assembly_MainPieceProductionCode_S.S. TUBE STEEL,Assembly_MainPieceProductionCode_SHIM,Assembly_MainPieceProductionCode_SHIM STACK,Assembly_MainPieceProductionCode_SS PLATE,Assembly_MainPieceProductionCode_STAIR STRINGER,Assembly_MainPieceProductionCode_STRINGER,Assembly_MainPieceProductionCode_TREAD,Assembly_MainPieceProductionCode_TRUSS CHORD,Assembly_MainPieceProductionCode_TUBE STEEL,Assembly_MainPieceProductionCode_TUBE STEEL BRIDGING,Assembly_MainPieceProductionCode_V-BRACE,Assembly_MainPieceProductionCode_WASHER,Assembly_MainPieceProductionCode_WT,Assembly_MainPieceProductionCode_WT SHEAR COLLECTOR,Assembly_MainPieceProductionCode_WT TOP TRUSS,Assembly_MainPieceProductionCode_WT TRUSS,Assembly_MainPartShape_AC,Assembly_MainPartShape_AFB,Assembly_MainPartShape_AHSS,Assembly_MainPartShape_AL,Assembly_MainPartShape_ALB,Assembly_MainPartShape_API,Assembly_MainPartShape_C,Assembly_MainPartShape_CHKD_PLT,Assembly_MainPartShape_FB,Assembly_MainPartShape_HP,Assembly_MainPartShape_HSS,Assembly_MainPartShape_HSSR,Assembly_MainPartShape_L,Assembly_MainPartShape_MC,Assembly_MainPartShape_PI,Assembly_MainPartShape_PL,Assembly_MainPartShape_S,Assembly_MainPartShape_W,Assembly_MainPartShape_WT,Assembly_MainPartDimension_1 1/2 x 1 1/2 x 1/4,Assembly_MainPartDimension_1 1/2SCH40,Assembly_MainPartDimension_1 1/4SCH40,Assembly_MainPartDimension_1 1/4SCH80,Assembly_MainPartDimension_1 3/4 x 1 3/4 x 1/4,Assembly_MainPartDimension_1 x 12 5/8 §,Assembly_MainPartDimension_1 x 15 1/2,Assembly_MainPartDimension_1 x 15 5/16,Assembly_MainPartDimension_1 x 17 11/16,Assembly_MainPartDimension_1 x 4 1/2,Assembly_MainPartDimension_1/2 x 10,Assembly_MainPartDimension_1/2 x 12,Assembly_MainPartDimension_1/2 x 15,Assembly_MainPartDimension_1/2 x 22,Assembly_MainPartDimension_1/2 x 29 §,Assembly_MainPartDimension_1/2 x 3,Assembly_MainPartDimension_1/2 x 3 1/2,Assembly_MainPartDimension_1/2 x 3 15/16 §,Assembly_MainPartDimension_1/2 x

In [8]:
# Split target column from dataset
y = df_reduce_encoded['TimeInSeconds_ThisWorkSegment']
X = df_reduce_encoded.drop(columns='TimeInSeconds_ThisWorkSegment')

In [9]:
# Preview the data
print("Features Shape:",X.shape)
print("Targe Shape:", y.shape)

Features Shape: (8407, 443)
Targe Shape: (8407,)


In [10]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
# Preview the data
print("Features Shape:",X.shape)
print("Targe Shape:", y.shape)

Features Shape: (8407, 443)
Targe Shape: (8407,)


In [12]:
# Display the shapes of the training and testing sets
print("Training Features Shape:", X_train.shape)
print("Training Target Shape:", y_train.shape)
print("Testing Features Shape:", X_test.shape)
print("Testing Target Shape:", y_test.shape)

Training Features Shape: (6305, 443)
Training Target Shape: (6305,)
Testing Features Shape: (2102, 443)
Testing Target Shape: (2102,)


In [13]:
# Creating StandardScaler instance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the Linear Regression Model

In [14]:
# Instantiate the linear regression model
model = LinearRegression()

In [15]:
# Fit the model to the training data
model.fit(X_train_scaled, y_train)

LinearRegression()

In [16]:
# Display the slope and y-intercept
print(f"Models slope: {model.coef_}")
print(f"Models y-intercept: {model.intercept_}")

Models slope: [ 1.34044966e+02 -1.04485522e+02 -4.30106860e+03  1.84965568e+04
 -2.47932683e+03  1.04079169e+03  1.48537269e+03  1.32607536e+16
  2.52322983e+16  2.55695407e+16  5.95753483e+15  6.13598805e+15
 -2.31691643e+15  1.15670084e+16  2.00251067e+16  1.82579692e+14
  2.50216396e+16  4.02206341e+15  2.42854644e+16  3.83932901e+16
  3.69708595e+14  7.37532823e+15  5.44157094e+15  2.46553221e+16
  5.53266326e+15  1.97686963e+16 -7.65522754e+16  2.62100061e+16
  3.63971385e+16  2.63977811e+16  3.96991781e+16 -6.85510036e+16
  9.08695026e+16  8.58125442e+15  1.36192412e+16  2.01246924e+15
  5.19332952e+16 -2.16314414e+16  3.48403925e+15 -6.93527500e+15
  3.28543854e+14  2.41656153e+14 -2.43330763e+16  9.87857030e+15
  7.68205859e+15  3.01434353e+13  2.32361494e+15  6.02438069e+13
  5.21851016e+13 -5.02803606e+15  1.93435994e+14  4.15978908e+14
  2.46139645e+13  5.79934908e+15  6.95312137e+15  1.23533360e+16
 -4.75420452e+16  1.66724404e+16  6.13598805e+15 -3.43487652e+16
  9.5117496

In [17]:
# Display the model's best fit line formula
print(f"Models formula: y = {model.intercept_}+{model.coef_[0]}X")

Models formula: y = 2568.7214602794897+134.04496648045796X


# Plot the Best Fit Line

In [18]:
# Predict the target values for the test set
##y_pred = model.predict(X_test_scaled)

In [19]:
# Predict the target values for the entire dataset
y_pred_all = model.predict(X)

C:\Users\bfletchall\AppData\Local\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [20]:
# Create a copy of the data
predicted_y_values = df_reduce_encoded.copy()

# Add column with predicted values
predicted_y_values["predicted_time"] = y_pred_all

# Display sample data
predicted_y_values.head()

,EPM_ProductionControlItemID,EPM_InstanceNumber,Assembly_MainPartLengthFt,Assembly_WeightEachLbs,Assembly_SurfaceAreaEachSqFt,Assembly_TotalQuantityInJob,Assembly_NumSmallParts,TimeInSeconds_ThisWorkSegment,Assembly_MainPieceProductionCode_ALUM ANGLE,Assembly_MainPieceProductionCode_ALUM BEAM,Assembly_MainPieceProductionCode_ALUM CHANNEL,Assembly_MainPieceProductionCode_ALUM CHKD PLATE,Assembly_MainPieceProductionCode_ALUM EMBED ANGLE,Assembly_MainPieceProductionCode_ALUM PIPE,Assembly_MainPieceProductionCode_ALUM PIPE LADDER,Assembly_MainPieceProductionCode_ALUM PIPE SLEEVE,Assembly_MainPieceProductionCode_ALUM PLATE,Assembly_MainPieceProductionCode_ALUM STRINGER,Assembly_MainPieceProductionCode_ALUM TUBE COLUMN,Assembly_MainPieceProductionCode_ANGLE,Assembly_MainPieceProductionCode_BEAM,Assembly_MainPieceProductionCode_BENT PLATE,Assembly_MainPieceProductionCode_BOLLARD,Assembly_MainPieceProductionCode_BRACE,Assembly_MainPieceProductionCode_BRACKET,Assembly_MainPieceProductionCode_Backer Bar,Assembly_MainPieceProductionCode_CHANNEL,Assembly_MainPieceProductionCode_CHK PLATE,Assembly_MainPieceProductionCode_COLUMN,Assembly_MainPieceProductionCode_CRANE GIRDER,Assembly_MainPieceProductionCode_DOOR PLATE,Assembly_MainPieceProductionCode_EMBED,Assembly_MainPieceProductionCode_EMBED ANGLE,Assembly_MainPieceProductionCode_EMBED PLATE,Assembly_MainPieceProductionCode_FRAME,Assembly_MainPieceProductionCode_FRAME_COLUMN,Assembly_MainPieceProductionCode_GALV. STRINGER,Assembly_MainPieceProductionCode_GUSSET PL,Assembly_MainPieceProductionCode_H BRACE,Assembly_MainPieceProductionCode_HSS STRUT,Assembly_MainPieceProductionCode_PIPE,Assembly_MainPieceProductionCode_PLATE,Assembly_MainPieceProductionCode_PLATE WASHER,Assembly_MainPieceProductionCode_RAIL,Assembly_MainPieceProductionCode_ROLLED TUBE STEEL,Assembly_MainPieceProductionCode_S.S. ANGLE,Assembly_MainPieceProductionCode_S.S. BENT PLATE,Assembly_MainPieceProductionCode_S.S. EMBED ANGLE,Assembly_MainPieceProductionCode_S.S. PLATE,Assembly_MainPieceProductionCode_S.S. PLATE LADDER,Assembly_MainPieceProductionCode_S.S. TUBE STEEL,Assembly_MainPieceProductionCode_SHIM,Assembly_MainPieceProductionCode_SHIM STACK,Assembly_MainPieceProductionCode_SS PLATE,Assembly_MainPieceProductionCode_STAIR STRINGER,Assembly_MainPieceProductionCode_STRINGER,Assembly_MainPieceProductionCode_TREAD,Assembly_MainPieceProductionCode_TRUSS CHORD,Assembly_MainPieceProductionCode_TUBE STEEL,Assembly_MainPieceProductionCode_TUBE STEEL BRIDGING,Assembly_MainPieceProductionCode_V-BRACE,Assembly_MainPieceProductionCode_WASHER,Assembly_MainPieceProductionCode_WT,Assembly_MainPieceProductionCode_WT SHEAR COLLECTOR,Assembly_MainPieceProductionCode_WT TOP TRUSS,Assembly_MainPieceProductionCode_WT TRUSS,Assembly_MainPartShape_AC,Assembly_MainPartShape_AFB,Assembly_MainPartShape_AHSS,Assembly_MainPartShape_AL,Assembly_MainPartShape_ALB,Assembly_MainPartShape_API,Assembly_MainPartShape_C,Assembly_MainPartShape_CHKD_PLT,Assembly_MainPartShape_FB,Assembly_MainPartShape_HP,Assembly_MainPartShape_HSS,Assembly_MainPartShape_HSSR,Assembly_MainPartShape_L,Assembly_MainPartShape_MC,Assembly_MainPartShape_PI,Assembly_MainPartShape_PL,Assembly_MainPartShape_S,Assembly_MainPartShape_W,Assembly_MainPartShape_WT,Assembly_MainPartDimension_1 1/2 x 1 1/2 x 1/4,Assembly_MainPartDimension_1 1/2SCH40,Assembly_MainPartDimension_1 1/4SCH40,Assembly_MainPartDimension_1 1/4SCH80,Assembly_MainPartDimension_1 3/4 x 1 3/4 x 1/4,Assembly_MainPartDimension_1 x 12 5/8 §,Assembly_MainPartDimension_1 x 15 1/2,Assembly_MainPartDimension_1 x 15 5/16,Assembly_MainPartDimension_1 x 17 11/16,Assembly_MainPartDimension_1 x 4 1/2,Assembly_MainPartDimension_1/2 x 10,Assembly_MainPartDimension_1/2 x 12,Assembly_MainPartDimension_1/2 x 15,Assembly_MainPartDimension_1/2 x 22,Assembly_MainPartDimension_1/2 x 29 §,Assembly_MainPartDimension_1/2 x 3,Assembly_MainPartDimension_1/2 x 3 1/2,Assembly_MainPartDimension_1/2 x 3 15/16 §,Assembly_MainPartDimension_1/2 x

In [21]:
import holoviews as hv

# Create a DataFrame for plotting the best fit line
plot_data = pd.DataFrame({'Assembly_WeightEachLbs': X_train_scaled[:,0], 'TimeInSeconds_ThisWorkSegment': y_train})

# Create a scatter plot
scatter_plot = plot_data.hvplot.scatter(
    x='Assembly_WeightEachLbs',
    y='TimeInSeconds_ThisWorkSegment',
    color='blue',
    title='Assembly_WeightEachLbs vs TimeInSeconds_ThisWorkSegment'
)

# Create best fit line
best_fit_line = hv.Curve((plot_data['Assembly_WeightEachLbs'], model.predict(X_train_scaled)), 'Assembly_WeightEachLbs', 'TimeInSeconds_ThisWorkSegment', label='Best Fit Line')

# Overlay scatter plot and best fit line
combined_plot = scatter_plot * best_fit_line

combined_plot

:Overlay
   .Scatter.I           :Scatter   [Assembly_WeightEachLbs]   (TimeInSeconds_ThisWorkSegment)
   .Curve.Best_Fit_Line :Curve   [Assembly_WeightEachLbs]   (TimeInSeconds_ThisWorkSegment)

In [22]:
# Create a line plot
best_fit_line = predicted_y_values.hvplot.scatter(
    x='Assembly_WeightEachLbs',
    y='TimeInSeconds_ThisWorkSegment',
    color='blue',
    title='Assembly_WeightEachLbs vs TimeInSeconds_ThisWorkSegment'
)

best_fit_line

:Scatter   [Assembly_WeightEachLbs]   (TimeInSeconds_ThisWorkSegment)

In [23]:
# Create a line plot
best_fit_line = predicted_y_values.hvplot.scatter(
    x='Assembly_MainPartLengthFt',
    y='TimeInSeconds_ThisWorkSegment',
    color='blue',
    title='Assembly_WeightEachLbs vs TimeInSeconds_ThisWorkSegment'
)

best_fit_line

:Scatter   [Assembly_MainPartLengthFt]   (TimeInSeconds_ThisWorkSegment)

In [24]:
# Create a line plot
best_fit_line = predicted_y_values.hvplot.scatter(
    x='Assembly_NumSmallParts',
    y='TimeInSeconds_ThisWorkSegment',
    color='blue',
    title='Assembly_WeightEachLbs vs TimeInSeconds_ThisWorkSegment'
)

best_fit_line

:Scatter   [Assembly_NumSmallParts]   (TimeInSeconds_ThisWorkSegment)

In [25]:
# Create a line plot
best_fit_line = predicted_y_values.hvplot.scatter(
    x='Assembly_WeightEachLbs',
    y='predicted_time',
    color='red',
    title='Predicted Time vs Actual Time'
)

best_fit_line

:Scatter   [Assembly_WeightEachLbs]   (predicted_time)

In [26]:
# Create a line plot
best_fit_line = predicted_y_values.hvplot.scatter(
    x='Assembly_NumSmallParts',
    y='predicted_time',
    color='red',
    title='Predicted Time vs Actual Time'
)

best_fit_line

:Scatter   [Assembly_NumSmallParts]   (predicted_time)

In [27]:
# Create a line plot
best_fit_line = predicted_y_values.hvplot.scatter(
    x='Assembly_MainPartLengthFt',
    y='predicted_time',
    color='red',
    title='Predicted Time vs Actual Time'
)

best_fit_line

:Scatter   [Assembly_MainPartLengthFt]   (predicted_time)

# Move away from linear regression
I do not see a liner relationship with individual features.  Move to Random Forest Regression to consider multiple features and the relationship between them.